In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
363,Doll Master is an example of a lousy horror fi...,negative
825,How can anyone argue the fact that Urban Cowbo...,positive
511,I'm sure this is a show no one is that familia...,positive
477,THHE2 is entertaining in that you'll laugh a l...,negative
990,"In this forgettable trifle, the 40-ish Norma S...",negative


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

Error during text normalization: 
**********************************************************************
  Resource wordnet not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('wordnet')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/wordnet

  Searched in:
    - 'C:\\Users\\yashk/nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\yashk\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\yashk/nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\yashk\\anaconda4\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\yashk\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
363,doll master example lousy horror film fallen s...,0
825,anyone argue fact urban cowboy was still is be...,1
511,i m sure show one familiar might think good it...,1
477,thhe entertaining laugh lot cringe probably sa...,0
990,forgettable trifle ish norma shearer plays flu...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/yashkhatri012/capstone-proj.mlflow')
dagshub.init(repo_owner='yashkhatri012', repo_name='capstone-proj', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\yashk\anaconda4\envs\atlas\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e265e7d0-3b25-4132-9d40-16366fc41a1b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=819bd52bcd12ea558ff0aa6a2d81d674e2cce457dbfa315dec253620fd20d914




Accessing as yashkhatri012

Initialized MLflow to track repo "yashkhatri012/capstone-proj"

Repository yashkhatri012/capstone-proj initialized!

2025/03/01 14:49:28 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7c3e1eb5c8ad40b99b49ab2c7baf0bc7', creation_time=1740820766648, experiment_id='0', last_update_time=1740820766648, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-01 14:54:14,206 - INFO - Starting MLflow run...
2025-03-01 14:54:15,299 - INFO - Logging preprocessing parameters...
2025-03-01 14:54:16,445 - INFO - Initializing Logistic Regression model...
2025-03-01 14:54:16,446 - INFO - Fitting the model...
2025-03-01 14:54:16,495 - INFO - Model training complete.
2025-03-01 14:54:16,495 - INFO - Logging model parameters...
2025-03-01 14:54:16,853 - INFO - Making predictions...
2025-03-01 14:54:16,855 - INFO - Calculating evaluation metrics...
2025-03-01 14:54:16,884 - INFO - Logging evaluation metrics...
2025-03-01 14:54:18,406 - INFO - Saving and logging the model...
2025/03/01 14:54:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-01 14:54:34,251 - INFO - Model training and logging completed in 18.95 seconds.
2025-03-01 14:54:34,252 - INFO - Accuracy: 0.656
2025-03-01 14:54:34,253 - INFO - Precision: 

🏃 View run nimble-colt-776 at: https://dagshub.com/yashkhatri012/capstone-proj.mlflow/#/experiments/0/runs/5ab9be58c59a4ce9bc82220ad4f5a454
🧪 View experiment at: https://dagshub.com/yashkhatri012/capstone-proj.mlflow/#/experiments/0
